In [1]:
!pip install meds-tab

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached hydra_optuna_sweeper-1.2.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached hydra_joblib_launcher-1.2.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached ml_mixins-0.1.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached optuna-2.10.1-py3-none-any.whl.metadata (15 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached nvidia_nccl_cu12-2.23.4-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached alembic-1.14.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached cliff-4.8.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached cmaes-0.11.1-py3-none-any.whl.metadata (18 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  U

In [2]:
#@title Download E-ICU demo
import os
from pathlib import Path
notebook_dir = os.getcwd()

ROOT_DIR=f"{notebook_dir}/work_dir/mimiciv_demo/"
# ROOT_DIR=f"{notebook_dir}/work_dir/eicu_demo/"
Path(ROOT_DIR).mkdir(parents=True, exist_ok=True)

MEDS_DIR = ROOT_DIR + "/meds/"
OUTPUT_TABULARIZATION_DIR= ROOT_DIR + "/tabularized/"
TASK_DIR = MEDS_DIR + "/task_labels"
TASK_NAME="mortality/in_icu/first_24h"
# TASK_NAME="los_in_hospital_first_48h"
OUTPUT_MODEL_DIR= ROOT_DIR + "/output/meds_tab/"

In [3]:
!meds-tab-describe input_dir={MEDS_DIR}/data output_dir={OUTPUT_TABULARIZATION_DIR}

2024-12-14 23:35:39.448 | INFO     | MEDS_tabular_automl.utils:stage_init:446 - Running describe_codes with the following configuration:
input_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data
output_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/
do_overwrite: false
seed: 1
tqdm: false
worker: 0
loguru_init: false
log_dir: ${output_dir}/.logs/
cache_dir: ${output_dir}/.cache
output_filepath: ${output_dir}/metadata/codes.parquet
name: describe_codes

2024-12-14 23:35:39.449 | DEBUG    | MEDS_tabular_automl.utils:stage_init:467 - Stage config:
input_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data
output_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/
do_overwrite: false
seed: 1
tqdm: false
worker: 0
loguru_init: false
log_dir: ${output_dir}/.logs/
cache_dir: ${output_dir}/.cache
output_filepath: ${output_dir}/metadata/codes.parquet
name: describe_codes

Paths: (check

In [4]:
# Define the window sizes and aggregations to generate features for
WINDOW_SIZES = "tabularization.window_sizes=[1d,30d,365d]"
AGGREGATIONS = "tabularization.aggs=[static/present,code/count,value/count,value/sum,value/sum_sqd,value/min,value/max]"

In [5]:
# !rm -rf /content/tabularized/tabularize/

In [6]:
!meds-tab-tabularize-static "input_dir=$MEDS_DIR/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" do_overwrite=False "$WINDOW_SIZES" "$AGGREGATIONS"

2024-12-14 23:35:41.366 | INFO     | MEDS_tabular_automl.utils:stage_init:446 - Running tabularize_static with the following configuration:
input_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data
output_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/
do_overwrite: false
seed: 1
tqdm: false
worker: 0
loguru_init: false
log_dir: ${output_dir}/.logs/
cache_dir: ${output_dir}/.cache
tabularization:
  filtered_code_metadata_fp: ${output_dir}/metadata/codes.parquet
  allowed_codes: null
  min_code_inclusion_count: 10
  min_code_inclusion_frequency: null
  min_correlation: null
  max_by_correlation: null
  max_included_codes: null
  window_sizes:
  - 1d
  - 30d
  - 365d
  aggs:
  - static/present
  - code/count
  - value/count
  - value/sum
  - value/sum_sqd
  - value/min
  - value/max
  _resolved_codes: ${filter_to_codes:${tabularization.filtered_code_metadata_fp},${tabularization.allowed_codes},${tabularization.min_code_inclusion_c

In [7]:
!meds-tab-tabularize-time-series --multirun "worker=range(0,2)" "hydra/launcher=joblib" "input_dir=$MEDS_DIR/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" do_overwrite=False  "$WINDOW_SIZES" "$AGGREGATIONS"

[2024-12-14 23:35:44,566][HYDRA] Joblib.Parallel(n_jobs=-1,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 2 jobs
[2024-12-14 23:35:44,566][HYDRA] Launching jobs, sweep output dir : /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/tabularized/.logs
[2024-12-14 23:35:44,566][HYDRA] 	#0 : worker=0 input_dir=/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data output_dir=/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/ do_overwrite=False tabularization.window_sizes=[1d,30d,365d] tabularization.aggs=[static/present,code/count,value/count,value/sum,value/sum_sqd,value/min,value/max]
[2024-12-14 23:35:44,566][HYDRA] 	#1 : worker=1 input_dir=/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data output_dir=/storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/ do_overwrite=False tabula

In [8]:
!meds-tab-cache-task "input_dir={MEDS_DIR}/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" "input_label_dir=$TASK_DIR/$TASK_NAME/" "task_name=$TASK_NAME" do_overwrite=False  "$WINDOW_SIZES" "$AGGREGATIONS"

2024-12-14 23:39:26.960 | INFO     | MEDS_tabular_automl.utils:stage_init:446 - Running cache_task with the following configuration:
input_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data
output_dir: /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/
do_overwrite: false
seed: 1
tqdm: false
worker: 0
loguru_init: false
log_dir: ${output_dir}/.logs/
cache_dir: ${output_dir}/.cache
tabularization:
  filtered_code_metadata_fp: ${output_dir}/metadata/codes.parquet
  allowed_codes: null
  min_code_inclusion_count: 10
  min_code_inclusion_frequency: null
  min_correlation: null
  max_by_correlation: null
  max_included_codes: null
  window_sizes:
  - 1d
  - 30d
  - 365d
  aggs:
  - static/present
  - code/count
  - value/count
  - value/sum
  - value/sum_sqd
  - value/min
  - value/max
  _resolved_codes: ${filter_to_codes:${tabularization.filtered_code_metadata_fp},${tabularization.allowed_codes},${tabularization.min_code_inclusion_count},$

In [9]:
!meds-tab-model --multirun "input_dir=$MEDS_DIR/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" "output_model_dir=$OUTPUT_MODEL_DIR/$TASK_NAME/" "task_name=$TASK_NAME" do_overwrite=False "hydra.sweeper.n_trials=10"  "$WINDOW_SIZES" "$AGGREGATIONS" "tabularization.min_code_inclusion_count=10"

[I 2024-12-14 23:39:44,970] A new study created in memory with name: no-name-caa18645-481d-4a43-b303-eeb4485e2bbb
[2024-12-14 23:39:44,971][HYDRA] Study name: no-name-caa18645-481d-4a43-b303-eeb4485e2bbb
[2024-12-14 23:39:44,971][HYDRA] Storage: None
[2024-12-14 23:39:44,971][HYDRA] Sampler: TPESampler
[2024-12-14 23:39:44,971][HYDRA] Directions: ['maximize']
[2024-12-14 23:39:44,974][HYDRA] Joblib.Parallel(n_jobs=-1,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 2 jobs
[2024-12-14 23:39:44,974][HYDRA] Launching jobs, sweep output dir : /storage/nassim/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/output/meds_tab/mortality/in_icu/first_24h/2024-12-14_23-39-43/hydra
[2024-12-14 23:39:44,974][HYDRA] 	#0 : +model_launcher.model.eta=0.22037365138299875 +model_launcher.model.lambda=0.06876318864824583 +model_launcher.model.alpha=0.1407563256868113 +model_launcher.model.subsampl